In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Question 1

In [2]:
def fixed_strike_asian_call(T=1,K=95,rho=-0.7,r=0.07,S0=100,V0=0.08411,sig=0.2,a=0.4,b=-4,n=100_000):
    m = int(np.sqrt(n))
    dt = T/m

    np.random.seed(420)
    mean = [0,0]
    cov = [[1,rho],[rho,1]]
    W1, W2 = np.random.multivariate_normal(mean,cov,(m,n)).T

    S = np.zeros((n,m+1))
    V = np.zeros((n,m+1))

    S[:,0] += S0
    V[:,0] += V0
    
    for i,_ in enumerate(np.linspace(dt,T,m)):
        V[:,i+1] = (a + b * np.where(V[:,i]>0, V[:,i], 0)) * dt + sig * np.sqrt(np.where(V[:,i]>0, V[:,i], 0) * dt) * W2[:,i] \
            + V[:,i]
        S[:,i+1] = r * S[:,i] * dt + np.sqrt(np.where(V[:,i]>0, V[:,i], 0) * dt) * S[:,i] * W1[:,i] + S[:,i]

    A1 = S.mean(axis=1)
    A2 = np.sqrt((S**2).mean(axis=1))

    c1 = np.maximum(A1 - K, 0)
    c1 = np.exp(-r * T) * c1.mean()

    c2 = np.maximum(A2 - K, 0)
    c2 = np.exp(-r * T) * c2.mean()

    return c1,c2

In [3]:
def asian_call_deltas(eps=0.1,T=1,K=95,rho=-0.7,r=0.07,S0=100,V0=0.08411,sig=0.2,a=0.4,b=-4,n=10_000):
    c1_plus,c2_plus = fixed_strike_asian_call(T,K,rho,r,S0+eps,V0,sig,a,b,n)
    c1_minus,c2_minus = fixed_strike_asian_call(T,K,rho,r,S0-eps,V0,sig,a,b,n)

    delta1 = (c1_plus - c1_minus) / 2 / eps
    delta2 = (c2_plus - c2_minus) / 2 / eps

    return delta1,delta2

In [4]:
data = []
for K in [95,105]:
    for T in [1,0.5,0.05]:
        res = {'K': K, 'T': T}
        c1,c2 = fixed_strike_asian_call(T,K)
        delta1, delta2 = asian_call_deltas(T=T,K=K)
        res['c1'] = c1
        res['delta1'] = delta1
        res['c2'] = c2
        res['delta2'] = delta2
        data.append(res)
df = pd.DataFrame(data)
df.set_index(['K','T'],inplace=True)

In [5]:
# df.to_latex()

'\\begin{tabular}{llrrrr}\n\\toprule\n &  & c1 & delta1 & c2 & delta2 \\\\\nK & T &  &  &  &  \\\\\n\\midrule\n\\multirow[t]{3}{*}{95} & 1.000000 & 11.242429 & 0.700359 & 11.770088 & 0.714202 \\\\\n & 0.500000 & 8.613678 & 0.726857 & 8.867034 & 0.735337 \\\\\n & 0.050000 & 5.278422 & 0.918823 & 5.308142 & 0.921078 \\\\\n\\cline{1-6}\n\\multirow[t]{3}{*}{105} & 1.000000 & 6.109117 & 0.499632 & 6.536802 & 0.513357 \\\\\n & 0.500000 & 3.395976 & 0.424650 & 3.576704 & 0.434284 \\\\\n & 0.050000 & 0.179706 & 0.110053 & 0.186539 & 0.112382 \\\\\n\\cline{1-6}\n\\bottomrule\n\\end{tabular}\n'

In [21]:
import numpy as np
import pandas as pd

def fixed_strike_asian_call(T=1, K=95, rho=-0.7, r=0.07, S0=100, V0=0.08411, sig=0.2, a=0.4, b=-4, n=100_000):
    m = 100  # Set the number of time steps explicitly
    dt = T / m  # Time step size

    np.random.seed(420)
    mean = [0, 0]
    cov = [[1, rho], [rho, 1]]
    W = np.random.multivariate_normal(mean, cov, (n, m))
    W1, W2 = W[:, :, 0], W[:, :, 1]  # Extract W1 and W2 from the generated random numbers

    S = np.zeros((n, m+1))
    V = np.zeros((n, m+1))

    S[:, 0] = S0
    V[:, 0] = V0
    
    for i in range(m):
        V[:, i+1] = (a + b * np.maximum(V[:, i], 0)) * dt + sig * np.sqrt(np.maximum(V[:, i], 0) * dt) * W2[:, i] + V[:, i]
        S[:, i+1] = r * S[:, i] * dt + np.sqrt(np.maximum(V[:, i], 0) * dt) * S[:, i] * W1[:, i] + S[:, i]

    A1 = S.mean(axis=1)
    A2 = np.sqrt((S**2).mean(axis=1))

    c1 = np.maximum(A1 - K, 0)
    c1 = np.exp(-r * T) * c1.mean()

    c2 = np.maximum(A2 - K, 0)
    c2 = np.exp(-r * T) * c2.mean()

    return c1, c2

def asian_call_deltas(eps=0.1, T=1, K=95, rho=-0.7, r=0.07, S0=100, V0=0.08411, sig=0.2, a=0.4, b=-4, n=10_000):
    c1_plus, c2_plus = fixed_strike_asian_call(T, K, rho, r, S0 + eps, V0, sig, a, b, n)
    c1_minus, c2_minus = fixed_strike_asian_call(T, K, rho, r, S0 - eps, V0, sig, a, b, n)

    delta1 = (c1_plus - c1_minus) / (2 * eps)
    delta2 = (c2_plus - c2_minus) / (2 * eps)

    return delta1, delta2

data = []
for K in [95, 105]:
    for T in [1, 0.5, 0.05]:
        res = {'K': K, 'T': T}
        c1, c2 = fixed_strike_asian_call(T, K)
        delta1, delta2 = asian_call_deltas(T=T, K=K)
        res['c1'] = c1
        res['delta1'] = delta1
        res['c2'] = c2
        res['delta2'] = delta2
        data.append(res)

df = pd.DataFrame(data)
df.set_index(['K', 'T'], inplace=True)

# Display the resulting DataFrame
print(df)

                 c1    delta1         c2    delta2
K   T                                             
95  1.00  11.274738  0.701549  11.804440  0.715360
    0.50   8.639931  0.727409   8.894698  0.737322
    0.05   5.289585  0.922788   5.319323  0.925045
105 1.00   6.118184  0.494503   6.547427  0.509046
    0.50   3.405057  0.423042   3.586593  0.431741
    0.05   0.181645  0.107991   0.188547  0.110222


# Question 2

In [6]:
def r_model(r0,z,a=6,b=0.6,sig=0.12,gam=2,dt=1/12):
    rt = (1 - a * dt) * np.maximum(r0, 0) + a * b * dt + sig * np.sqrt(dt) * np.maximum(r0, 0)**gam * z
    return rt

In [7]:
def bond_monte_carlo(T=5,r0=0.05,a=6,b=0.6,u=1,gam=2,lam=0.3,mu=14.6,sig=3.5,D=1.8e9,FV=1,Jump=True,dt=0.005,N=10_000,seed=420):
    M = int(T / dt)

    np.random.seed(seed)
    z = np.random.normal(size=(N,M))
    r = np.zeros((N,M+1))
    r[:,0] += r0

    for j in range(M):
        r[:,j+1] = r_model(r[:,j],z[:,j],a,b,u,gam,dt)

    R = np.sum(r,axis=1) * dt

    PD = FV * np.exp(-R)
    PD = PD.mean()

    if Jump:
        Jt = np.random.exponential(1/T/lam, size=(N,M))
        Jt = Jt.cumsum(axis=1)
        column_T = np.where(Jt.min(axis=0) > T)[0][0]
        Jt = Jt[:,:column_T]

        S = np.zeros(N)

        for j in range(Jt.shape[1]):
            ind = Jt[:,j] < T
            lnX = np.random.normal(mu,sig,size=ind.sum())
            S[ind] += np.exp(lnX)
        
        

        CAT_flag = np.where(S < D,1,0)
        prob_CAT = (S>=D).sum() / N
        CAT = FV * np.exp(-R) * CAT_flag
        CAT = CAT.mean()

        return PD,prob_CAT,CAT
    
    return PD

In [8]:
PD,prob_CAT,CAT = bond_monte_carlo(u=1)
PD

0.05504178668993333

In [9]:
prob_CAT

0.2201

In [10]:
CAT

0.042958240659996795

# Question 3

In [22]:
def lookback_option(K=100,S0=100,T=5,r=0.05,sig=0.35,N=100_000):
    m = int(np.sqrt(N))
    dt = T/m
    mu = r - sig**2/2
    
    Z = np.zeros((N,m))
    np.random.seed(420)
    Z = np.random.normal(size=(N,m))

    S = np.zeros((N,m+1))
    S[:,0] = S0
    S = S0 * np.exp(mu*dt + sig*np.sqrt(dt)*Z).cumprod(axis=1)

    tau = np.zeros(N)
    tau_L = np.zeros(N)
    tau_U = np.zeros(N)
    payoff = np.zeros(N)

    for i in range(1,m):
        t = dt * i

        L = 50 * np.exp(0.138629 * t)
        U = 200 - L

        ind = (tau_L == 0) & (tau_U == 0)
        L_ind = ind & (S[:,i] <= L)
        U_ind = ind & (S[:,i] >= U)

        tau_L[L_ind] = t
        tau_U[U_ind] = t

        tau[L_ind] = t
        tau[U_ind] = t

        payoff[L_ind] = np.maximum(K - S[L_ind,i], 0)
        payoff[U_ind] = np.maximum(S[U_ind,i] - K, 0)

    prob_exercise  = len(tau[tau>0]) / N
    prob_U_exercise = len(tau_U[tau_U>0]) / len(tau[tau>0])

    return payoff.mean(),prob_exercise,prob_U_exercise


In [23]:
lookback_option()

(41.07948414604784, 1.0, 0.55576)

# Question 4

In [13]:
def numerix_prepayment_model(prev_r, R, PV0, prev_PV, t):
    SG = min(1, t / 30)
    BU = 0.3 + 0.7 * (prev_PV / PV0)
    RI = 0.28 + 0.14 * (np.arctan(-8.57 + 430 * (R - prev_r)))
    CPR =  RI * BU * SG
    return CPR

In [14]:
def MBS_pricing(PV0=1_000_000,WAC=0.064,r0=0.065,kap=5.8,r_mu=0.055,sig=1,gam=2,T=30,N=1000,dt=1/120,seed=420):
    mortgage_rate = WAC / 12
    M = int(T / dt)
    div = int(1 / dt / 12)

    PV = np.zeros((N,T*12+1))
    r  = np.zeros((N,M+1))
    PV[:,0] = PV0
    r[:,0]  = r0

    IO = np.zeros((N,T*12))
    PO = np.zeros((N,T*12))

    np.random.seed(seed)
    z = np.random.normal(size=(N,M))

    Price = np.zeros(N)
    for t in range(M):
        r[:,t+1] = r_model(r[:,t],z[:,t],kap,r_mu,sig,gam,dt)
        if (t + 1) % div == 0:
            m = int((t + 1) / div) - 1
            IP = PV[:,m] * mortgage_rate
            MP = IP / (1 - 1 / (1 + mortgage_rate)**(T * 12 - m + 1))
            SP = MP - IP

            zcb_price = bond_monte_carlo(10,r[:,t],kap,r_mu,sig,gam,0,0,0,0,1,False,0.5,N,seed)
            prev_r = -np.log(zcb_price) / 10
            CPR = numerix_prepayment_model(prev_r,WAC,PV0,PV[:,m],m+1)

            SMM = 1 - (1 - CPR)**(1 / 12)
            PP = (PV[:,m] - SP) * SMM
            c = MP + PP
            TPP = SP + PP

            R = np.sum(r[:,:t+1],axis=1) * dt
            d = np.exp(-R)

            IO[:,m] = d * IP
            PO[:,m] = d * TPP

            Price += d * c
            PV[:,m+1] = PV[:,m] - TPP

    return Price.mean(), IO.sum(axis=1).mean(), PO.sum(axis=1).mean()

In [15]:
MBS_pricing()

(1027688.3647943181, 211869.9289054259, 815818.4358888922)

In [16]:
def MBS_duration(eps=0.01,PV0=1_000_000,WAC=0.064,r0=0.065,kap=5.8,r_mu=0.055,sig=1,gam=2,T=30,N=1000,dt=1/120,seed=420):
    _,IO_plus,_  = MBS_pricing(PV0,WAC+eps,r0,kap,r_mu,sig,gam,T,N,dt,seed)
    _,IO_minus,_ = MBS_pricing(PV0,WAC-eps,r0,kap,r_mu,sig,gam,T,N,dt,seed)
    print(IO_plus,IO_minus)

    IO_duration = (IO_plus - IO_minus) / 2 / eps / PV0
    return IO_duration

In [17]:
data = []
for a in np.linspace(0.03,0.09,7):
    res  = {'a': a}
    _,IO,_ = MBS_pricing(r_mu=a)
    IO_duration = MBS_duration(r_mu=a)
    res['IO Price'] = IO
    res['IO Duration'] = IO_duration
    data.append(res)
df = pd.DataFrame(data)
df.set_index('a',inplace=True)
df

262062.84483832997 191506.9633403454
254282.51141674598 187255.84958297774
247121.4621752328 185020.4829972867
240564.5693954499 190202.03679378785
234620.5189020111 248809.97077051323
229409.41817439368 305777.64288609364
225254.57602246714 312239.62575295434


,IO Price,IO Duration
a,,
0.03,226343.224410,3.527794
0.04,220001.172933,3.351333
0.05,214370.245150,3.105049
0.06,209637.595516,2.518127
0.07,206351.017081,-0.709473
0.08,207045.973336,-3.818411
0.09,231534.799038,-4.349252


In [18]:
!jupyter nbconvert --to script MFE405_Final.ipynb

[NbConvertApp] Converting notebook MFE405_Final.ipynb to script
[NbConvertApp] Writing 6723 bytes to MFE405_Final.py
